In [35]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [36]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_structured_chat_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

from langchain.tools import tool

In [37]:
class Extract_toolArgs(BaseModel):
    a: str = Field(description="Company name")
    b: str = Field(description="item")
    c: str = Field(description="Year_1")
    d: str = Field(description="Year_2")


In [38]:
years  = [2014,2015,2016,2017,2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [ ]:
class GrowthToolArgs(BaseModel):
    sales_data : list = Field(description="list of dictionaries with keys as year and net sales data point as values")


In [ ]:
import json
@tool(args_schema=Extract_toolArgs)
def Extract_Tool(a: str, b: str, c:str, d:str) -> list:
    """extracts specific data for a company from Year_1 to Year_2."""
    with open("data_soo.json","r") as json_file:
        data = json.load(json_file)
    a = a.upper()
    b = b.lower()
    json_data = next(item for item in data[a] if item["Item"] == b)
    cash_data = []
    if int(c) > int(d): c, d = d, c
    for year in years:
        if year >= int(c) and year <= int(d):
            cash_data.append({year: json_data.get(str(year))})
    # sales_year_1 = json_data.get(c)
    # sales_year_2 = json_data.get(d)
    if len(cash_data) == 0:
        raise ValueError(f"Data for the years {c} to {d} is not available.")
    
    return cash_data




In [ ]:
@tool(args_schema=GrowthToolArgs)
def Growth_Tool(sales_data:list) -> list:
    """
    Calculates year-on-year (YoY) growth for 'Net Sales' for a specific company. This function receives a list of dictionaries as data,
    where the key is year and value is the data point. 
    
    The tool automatically fetches data points of corresponding year and computes the
    percentage growth for that year compared to the previous year.
    
    
    Arguments:
    - sales_data: [{<year1>: <net sales1>, <year2>: <net sales2>... }]

    """
    # Hardcode the item to "Net Sales"
    # item = "Net sales"
    
    # # Call Extract_Tool to get sales data
    # try:
    #     sales_data = Extract_Tool(a=company, b=item, c=year_1, d=year_2)
    # except ValueError as e:
    #     return str(e)
    
    # Ensure there is sufficient data for year-on-year calculation
    if len(sales_data) < 2:
        return "Not enough data to calculate year-on-year sales growth."
    
    # Calculate YoY growth
    
    yoy_growth = []

    for i in sales_data.items():
        current_year_sales = i.value()
        previous_year = int(i.key() - 1)
        try:
            previous_year_sales = previous_year.value()
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(f"Year {i}: {growth:.2f}%")

        except KeyError:
            print('here')
            continue
        
    # for i in range(1, len(sales_data)):
    #     previous_year_sales = sales_data[i - 1]
    #     current_year_sales = sales_data[i]
    #     if previous_year_sales is None or current_year_sales is None:
    #         yoy_growth.append(f"Year {start_year + i}: Data missing")
    #     else:
    #         growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
    #         yoy_growth.append(f"Year {start_year + i}: {growth:.2f}%")
    
    return "\n".join(yoy_growth)


In [42]:
tools = [Extract_Tool,Growth_Tool]

In [43]:
# class CalculateKpIs(BaseModel):
#     a: str = Field(description="Company name")
#     b: str = Field(description="item")
#     c: list = Field(description="list of revenue data points")

# @tool(args_schema=CalculateKpIs)
# def CalculateKpIs(a: str, b: str, c:list) -> list:
#     """calculates kpis according to company and item name"""

In [44]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key_string)

In [45]:
prompt = hub.pull("hwchase17/openai-tools-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [46]:
# from langchain.prompts import PromptTemplate

# prompt = PromptTemplate(
#     template="""
# You are an AI assistant skilled in calling tools to perform calculations and answer questions.
# Here are the tools you can use:
# {tool_descriptions}

# When asked a question, decide which tool to use and call it appropriately.
# Always respond with your reasoning and the final answer.

# Use the following format:
# Thought: I should use [Tool Name]
# Action: [Tool Name] <arguments>
# Observation: <output of the tool>

# If no tools are needed, provide a direct answer.

# {agent_scratchpad}
# """,
#     input_variables=["tool_descriptions", "agent_scratchpad"],
# )


In [47]:
from pprint import pprint
pprint(prompt)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [48]:
agent = create_tool_calling_agent(
    llm=llm,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt,  # Prompt template to guide the agent's responses
)

In [49]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [50]:
response = agent_executor.invoke({"input": " extract net sales and then calculate year on year sales growth of APPLE from 2019 to 2023"})



> Entering new AgentExecutor chain...

Invoking: `Extract_Tool` with `{'a': 'APPLE', 'b': 'Net Sales', 'c': '2019', 'd': '2023'}`


[260174, 274515, 365817, 394328, 383285]
Invoking: `Growth_Tool` with `{'sales_data': [260174, 274515, 365817, 394328, 383285]}`




NameError: name 'year_1' is not defined

In [ ]:
print(response)

{'input': ' extract net sales of APPLE from 2019 to 2023', 'output': 'The net sales for Apple from 2019 to 2023 are as follows (in million USD):\n\n- 2019: $260,174 million\n- 2020: $274,515 million\n- 2021: $365,817 million\n- 2022: $394,328 million\n- 2023: $383,285 million'}
